In [1]:
import meerkat as mk


from meerkat.ops import groupby
from meerkat.ops.clusterby import clusterby, _clusterby
from meerkat.ops import generate_candidates


In [2]:
clusterby_intermediate = mk.DataPanel.read("imagenette_clip_embedded_full.dp")
clusters = _clusterby(clusterby_intermediate, [".emb"])


classes = list(set(clusterby_intermediate["label"].data.values))

sh = clusters.data.shape


correct_col = mk.DataPanel({"class" : [c for c in classes], "label" : [f"a photo of a {c}" for c in classes]})
mk.embed(correct_col, "label", modality = "text", out_col = ".emb", num_workers = 0)







  0%|          | 0/1 [00:00<?, ?it/s]

,class (PandasSeriesColumn),label (PandasSeriesColumn),.emb (NumpyArrayColumn)
0,english springer spaniel,a photo of a english springer spaniel,"np.ndarray(shape=(512,))"
1,golf ball,a photo of a golf ball,"np.ndarray(shape=(512,))"
2,tench,a photo of a tench,"np.ndarray(shape=(512,))"
3,chainsaw,a photo of a chainsaw,"np.ndarray(shape=(512,))"
4,church,a photo of a church,"np.ndarray(shape=(512,))"
5,parachute,a photo of a parachute,"np.ndarray(shape=(512,))"
6,garbage truck,a photo of a garbage truck,"np.ndarray(shape=(512,))"
7,gas pump,a photo of a gas pump,"np.ndarray(shape=(512,))"
8,cassette player,a photo of a cassette player,"np.ndarray(shape=(512,))"
9,french horn,a photo of a french horn,"np.ndarray(shape=(512,))"


In [3]:
clusterby_intermediate[".emb"].shape, correct_col[".emb"].shape

import numpy as np
import torch
embedded_images = clusterby_intermediate[".emb"]
embedding_label = correct_col[".emb"]




for i in range(len(embedded_images)):
    im = torch.Tensor(embedded_images[i])
    im /= im.norm(dim = -1, keepdim = True)
    embedded_images[i] = im
# d = {}

t_embedded_ims = torch.Tensor(embedded_images)

scores = torch.zeros((len(embedded_images), len(classes)))
for i in range(embedding_label.shape[0]):

    emb = correct_col[i][".emb"] 

    t = torch.Tensor(emb)
    t /= t.norm(dim = -1, keepdim = True)

    out = (t_embedded_ims * t).sum(axis = 1)

    scores[:, i] = out



preds = scores.softmax(dim = -1).argmax(dim = -1)




clusterby_intermediate[".pred_idx"] = preds
print([classes[i] for i in preds])




    
    # print(t.shape)

    # print(emb.shape)

    # n_ = torch.linalg.norm(emb, keepdim = True)
   
    # print(n_)

    # print((embedded_images - emb).shape)
    # print(l)
    # print("Embedding, emb.shape)
    # d[label] = emb

# dp_embedded_labels = mk.DataPanel(d)
    

# embedded_images.shape, dp_embedded_labels.shape

['cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'chainsaw', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'chainsaw', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'garbage truck', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassette player', 'cassett

/var/folders/gk/4qpy_m5j67d9h1h0g9j07wzr0000gn/T/ipykernel_77940/490106071.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  t_embedded_ims = torch.Tensor(embedded_images)


In [4]:
c = classes[6]
selected_rows = clusterby_intermediate.lz["label"] != clusterby_intermediate.lz[".pred_c"] 

clusterby_intermediate.lz[selected_rows]["img", "label", ".pred_c"]

KeyError: 'Column `.pred_c` does not exist.'

In [ ]:
import pandas as pd
def get_wiki_words(top_k: int = 1e5, eng_only: bool = True):
    df = pd.read_csv(
        "https://github.com/IlyaSemenov/wikipedia-word-frequency/raw/master/results/enwiki-20190320-words-frequency.txt",
        delimiter=" ",
        names=["word", "frequency"],
    )

    if eng_only:
        import nltk
        from nltk.corpus import words

        nltk.download("words")

        eng_words = words.words()
        eng_df = pd.DataFrame({"word": eng_words})
        df = df.merge(eng_df, how="inner", on="word")

    df = df.sort_values("frequency", ascending=False)
    df = df.drop_duplicates(subset=["word"])
    return mk.DataPanel.from_pandas(df.iloc[: int(top_k)])

In [8]:
template_descriptions = [
    "a photo of a {} [MASK] [MASK] ",
    "a photo of [MASK] {} ",
    "a photo of [MASK] {} ",
    "a photo of a {} [MASK] ",
    "a {} [MASK]"
]
import torch 
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
panel = generate_candidates.generate_candidate_descriptions(templates=template_descriptions, device=device, score_with_gpt=True)



[nltk_data] Downloading package words to
[nltk_data]     /Users/samrandall/nltk_data...
[nltk_data]   Package words is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/625 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

  0%|          | 0/30000 [00:00<?, ?it/s]

In [9]:
panel.write("bert_candidates_embeddings_gpt.dp")

In [14]:

mk.embed(panel, "output_phrase", modality = "text", out_col = ".emb", num_workers = 0)

  0%|          | 0/235 [00:00<?, ?it/s]